Libraries imported


In [3]:
#IMPORT LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("All libraries imported successfully")

All libraries imported successfully


loads three key COVID-19 datasets from the Johns Hopkins University repository.

In [4]:
base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"


# Global confirmed cases
confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

# Global deaths
deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

# Global number of people who recovered
recovered_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Load all three datasets 
confirmed_df = pd.read_csv(confirmed_url)
deaths_df = pd.read_csv(deaths_url)  
recovered_df = pd.read_csv(recovered_url)  

# Display the first few rows of each
print("       CONFIRMED CASES      ")
print(confirmed_df.head())
print(f"\nShape: {confirmed_df.shape}")
print(f"Columns: {confirmed_df.columns.tolist()[:5]}...")  

print("          DEATHS          ")
print(deaths_df.head())
print(f"\nShape: {deaths_df.shape}")
print(f"Columns: {deaths_df.columns.tolist()[:5]}...")  

print("          RECOVERED             ")
print(recovered_df.head())
print(f"\nShape: {recovered_df.shape}")
print(f"Columns: {recovered_df.columns.tolist()[:5]}...")  

       CONFIRMED CASES      
  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  2/28/23  3/1/23  3/2/23  3/3/23  \
0        0        0        0        0  ...   209322  209340  209358  209362   
1        0        0        0        0  ...   334391  334408  334408  334427   
2        0        0        0        0  ...   271441  271448  271463  271469   
3        0        0        0        0  ...    47866   47875   47875   47875   
4        0        0        0        0  ...   105255  105277  105277  105277   

   3/4/23  3/5/23  3/6/23  3/7/23  3/8/23  3/9/23  

This code systematically analyzes and compares the three COVID-19 datasets (confirmed cases, deaths, and recoveries). It loops through each dataset to print key information including its dimensions, memory usage, sample rows, column structure, missing values, and geographic coverage. This exploration helps understand each dataset's structure, identify data quality issues, and verify that all three datasets are properly loaded and consistent with each other before proceeding with deeper analysis.



In [5]:
# INITIAL DATA EXPLORATION
print()
print("INITIAL DATA EXPLORATION")
print()


# Dictionary of all datasets for easy looping
datasets = {
    "Confirmed Cases": confirmed_df,
    "Deaths": deaths_df,
    "Recovered": recovered_df
}

for name, df in datasets.items():
    print()
    print(f"EXPLORING: {name}")
    print()
    
    print(f"\nShape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print("\nFirst 3 rows:")
    print(df.head(3))
    
    print("\nColumns (first 10):")
    print(df.columns.tolist()[:10])
    
    print("\nMissing values (first 5 columns):")
    print(df.isnull().sum().head())
    
    print(f"\nUnique countries: {df['Country/Region'].nunique()}")


INITIAL DATA EXPLORATION


EXPLORING: Confirmed Cases


Shape: (289, 1147)
Memory usage: 2.55 MB

First 3 rows:
  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  2/28/23  3/1/23  3/2/23  3/3/23  \
0        0        0        0        0  ...   209322  209340  209358  209362   
1        0        0        0        0  ...   334391  334408  334408  334427   
2        0        0        0        0  ...   271441  271448  271463  271469   

   3/4/23  3/5/23  3/6/23  3/7/23  3/8/23  3/9/23  
0  209369  209390  209406  209436  209451  209451  
1  334427  334427  334427  334427  334443  334457  
2  271469  271477  271477  271490  271494  271496  

[3 rows x 1147 columns]

Columns (first 10):
['Province/State', 

Performs systematic data cleaning on your three COVID-19 datasets by first creating copies to avoid modifying the originals, then iterating through each dataset to handle missing values by filling empty province/state entries with "National" and converting all date-column nulls to zeros, while also checking for duplicate rows. After cleaning, it updates the original DataFrames with the processed versions and provides a summary showing the before/after shapes and missing value counts for each dataset, ensuring all three are consistently formatted and ready for analysis.

In [6]:
print()
print("DATA CLEANING & PREPROCESSING")
print()

datasets = {
    "Confirmed Cases": confirmed_df.copy(),
    "Deaths": deaths_df.copy(),
    "Recovered": recovered_df.copy()
}

cleaned_datasets = {}

for name, df in datasets.items():
    print(f"\n{'='*40}")
    print(f"CLEANING: {name}")
    print('='*40)
    
    # Store original shape
    original_shape = df.shape
    
    # 1. Check missing values
    missing_before = df.isnull().sum().sum()
    print(f"Missing values before: {missing_before}")
    
    # 2. Check duplicates (should be 0 for this data structure)
    duplicates = df.duplicated().sum()
    print(f"Duplicate rows: {duplicates}")
    
    # 3. Handle Province/State missing values
    df['Province/State'] = df['Province/State'].fillna('National')
    
    # 4. Fill numeric (date) columns with 0 for missing values
    date_columns = df.columns[4:]  # Skip first 4 geo columns
    df[date_columns] = df[date_columns].fillna(0).astype(int)
    
    # 5. Verify no missing values remain
    missing_after = df.isnull().sum().sum()
    print(f"Missing values after: {missing_after}")
    
    # Store cleaned dataset
    cleaned_datasets[name] = df
    
    print(f"Cleaned {name}: {original_shape} → {df.shape}")

print()
print("DATA CLEANING SUMMARY")
print()

# Update original DataFrames
confirmed_df = cleaned_datasets["Confirmed Cases"]
deaths_df = cleaned_datasets["Deaths"]
recovered_df = cleaned_datasets["Recovered"]

print("All datasets cleaned and ready for analysis")


DATA CLEANING & PREPROCESSING


CLEANING: Confirmed Cases
Missing values before: 202
Duplicate rows: 0
Missing values after: 4
Cleaned Confirmed Cases: (289, 1147) → (289, 1147)

CLEANING: Deaths
Missing values before: 202
Duplicate rows: 0
Missing values after: 4
Cleaned Deaths: (289, 1147) → (289, 1147)

CLEANING: Recovered
Missing values before: 201
Duplicate rows: 0
Missing values after: 2
Cleaned Recovered: (274, 1147) → (274, 1147)

DATA CLEANING SUMMARY

All datasets cleaned and ready for analysis


 Takes the three separate COVID-19 spreadsheets and combines them into one big spreadsheet where every row shows the cases, deaths, and recoveries for each country on each day, making it much easier to analyze how the pandemic changed over time.

In [7]:
# create combined data set

# Function to melt wide format to long format
def melt_covid_data(df, value_name="Cases"):
    """Convert wide format (many date columns) to long format"""
    melted = df.melt(
        id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
        var_name='Date',
        value_name=value_name
    )
    melted['Date'] = pd.to_datetime(melted['Date'])
    return melted

# Create long format datasets
confirmed_long = melt_covid_data(confirmed_df, "Confirmed")

deaths_long = melt_covid_data(deaths_df, "Deaths")

recovered_long = melt_covid_data(recovered_df, "Recovered")

# Merge all three
df = confirmed_long.merge(
    deaths_long, 
    on=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date']
).merge(
    recovered_long,
    on=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date']
)

print(f"Combined dataset created: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")

Combined dataset created: (306324, 8)
Date range: 2020-01-22 00:00:00 to 2023-03-09 00:00:00


 Calculates and displays the overall COVID-19 pandemic statistics by adding up all the confirmed cases, deaths, and recoveries from every country and date in the dataset, then calculates what percentage of total cases resulted in deaths (mortality rate) and what percentage led to recoveries (recovery rate), giving a high-level summary of the global impact of the pandemic.

In [10]:
#EXPLORATORY DATA ANALYSIS (EDA)
print()
print("EXPLORATORY DATA ANALYSIS")
print()

# Global statistics
print("\nGlobal COVID-19 Statistics:")
print()
total_cases = df['Confirmed'].sum()
total_deaths = df['Deaths'].sum()
total_recovered = df['Recovered'].sum()
global_mortality = (total_deaths / total_cases * 100)
global_recovery = (total_recovered / total_cases * 100)

print(f"Total Confirmed Cases: {total_cases:,}")
print(f"Total Deaths: {total_deaths:,}")
print(f"Total Recovered: {total_recovered:,}")
print(f"Global Mortality Rate: {global_mortality:.2f}%")
print(f"Global Recovery Rate: {global_recovery:.2f}%")


EXPLORATORY DATA ANALYSIS


Global COVID-19 Statistics:

Total Confirmed Cases: 314,458,609,242
Total Deaths: 4,385,973,999
Total Recovered: 23,208,807,963
Global Mortality Rate: 1.39%
Global Recovery Rate: 7.38%


Finds the 10 countries with the highest total COVID-19 cases, shows their total numbers of confirmed cases, deaths, and recoveries, and calculates what percentage of their cases resulted in deaths, displaying all this information in a ranked table.

In [11]:
# Top 10 countries by total cases
print("\nTop 10 Countries by Total Cases:")
print("-" * 40)
top_countries = df.groupby('Country/Region').agg({
    'Confirmed': 'max',
    'Deaths': 'max',
    'Recovered': 'max'
}).sort_values('Confirmed', ascending=False).head(10)
top_countries['Mortality_Rate'] = (top_countries['Deaths'] / top_countries['Confirmed'] * 100).round(2)
print(top_countries)


Top 10 Countries by Total Cases:
----------------------------------------
                Confirmed   Deaths  Recovered  Mortality_Rate
Country/Region                                               
US              103802702  1123836    6298082            1.08
India            44690738   530779   30974748            1.19
France           38618509   161512     342647            0.42
Germany          38249060   168935    3659260            0.44
Brazil           37081209   699276   17771228            1.89
Japan            33320438    72997     852451            0.22
Korea, South     30615522    34093     180719            0.11
Italy            25603510   188322    4144608            0.74
United Kingdom   24425309   219948       8322            0.90
Russia           22075858   388478    5609682            1.76


The code analyzes COVID-19 trends in 2021 by grouping the data month by month, adding up all the new cases, deaths, and recoveries that occurred each month, and displaying these monthly totals in a table to show how the pandemic progressed throughout that year.

In [ ]:
# Monthly trends
print("\nMonthly COVID-19 Trends (2021):")
print("-" * 40)
monthly_2021 = df[df['Year'] == 2021].groupby('Month').agg({
    'Confirmed': 'sum',
    'Deaths': 'sum',
    'Recovered': 'sum'
})
print(monthly_2021)

 Calculates and displays average COVID-19 statistics for each country by grouping the data by country, then computing the average number of confirmed cases, deaths, mortality rate, and recovery rate per entry, and shows the results for the first 10 countries in a formatted table.

In [12]:
df['Mortality_Rate'] = (df['Deaths'] / df['Confirmed'] * 100).round(2)
df['Recovery_Rate'] = (df['Recovered'] / df['Confirmed'] * 100).round(2)

# Country-wise statistics
print("\nCountry-wise Average Statistics:")
print("-" * 40)
country_stats = df.groupby('Country/Region').agg({
    'Confirmed': 'mean',
    'Deaths': 'mean',
    'Mortality_Rate': 'mean',
    'Recovery_Rate': 'mean'
}).round(2)
print(country_stats.head(10))


Country-wise Average Statistics:
----------------------------------------
                      Confirmed    Deaths  Mortality_Rate  Recovery_Rate
Country/Region                                                          
Afghanistan           113725.69   4743.16            3.89          29.48
Albania               162347.03   2174.44            1.85          30.30
Algeria               159878.96   4288.08            3.40          29.29
Andorra                21476.40    111.28            1.46          38.04
Angola                 52515.49   1077.72            2.83          26.82
Antarctica                 4.34      0.00            0.00           0.00
Antigua and Barbuda     3771.00     70.25            3.00          35.01
Argentina            4921682.35  79647.55            2.07          32.18
Armenia               249773.69   4991.59            1.87          34.40
Australia             383734.13    611.42            1.08          37.51


Creates a relationship table showing how six different COVID-19 statistics—total cases, deaths, recoveries, active cases, death percentage, and recovery percentage—move together or in opposite directions, helping identify patterns like whether countries with higher recovery rates tend to have lower death rates.

In [14]:
# Correlation analysis
print("\nCorrelation Matrix:")
print("-" * 40)

#  calculate Active cases and rates 
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']
df['Mortality_Rate'] = (df['Deaths'] / df['Confirmed'] * 100).round(2)
df['Recovery_Rate'] = (df['Recovered'] / df['Confirmed'] * 100).round(2)

# correlation with all columns
correlation_cols = ['Confirmed', 'Deaths', 'Recovered', 'Active', 'Mortality_Rate', 'Recovery_Rate']
correlation_matrix = df[correlation_cols].corr()
print(correlation_matrix)


Correlation Matrix:
----------------------------------------
                Confirmed    Deaths  Recovered    Active  Mortality_Rate  \
Confirmed        1.000000  0.887649   0.136171  0.989605       -0.011210   
Deaths           0.887649  1.000000   0.204508  0.863544       -0.000572   
Recovered        0.136171  0.204508   1.000000 -0.007592       -0.000064   
Active           0.989605  0.863544  -0.007592  1.000000       -0.011427   
Mortality_Rate  -0.011210 -0.000572  -0.000064 -0.011427        1.000000   
Recovery_Rate   -0.128101 -0.106776   0.145618 -0.150319       -0.017057   

                Recovery_Rate  
Confirmed           -0.128101  
Deaths              -0.106776  
Recovered            0.145618  
Active              -0.150319  
Mortality_Rate      -0.017057  
Recovery_Rate        1.000000  


In [ ]:
# SECTION 6: DATA VISUALIZATIONS
print("\n" + "="*60)
print("GENERATING VISUALIZATIONS")
print("="*60)

# Make sure you have matplotlib imported
import matplotlib.pyplot as plt
import seaborn as sns

# Figure 1: Global Trends Over Time
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('COVID-19 Global Trends Analysis', fontsize=20, fontweight='bold')

# Subplot 1: Total cases over time
daily_global = df.groupby('Date').agg({
    'Confirmed': 'sum',
    'Deaths': 'sum',
    'Recovered': 'sum',
    'Active': 'sum'
})

axes[0, 0].plot(daily_global.index, daily_global['Confirmed'], 
                linewidth=2, label='Confirmed', color='#3498db')
axes[0, 0].plot(daily_global.index, daily_global['Deaths'], 
                linewidth=2, label='Deaths', color='#e74c3c')
axes[0, 0].plot(daily_global.index, daily_global['Recovered'], 
                linewidth=2, label='Recovered', color='#2ecc71')
axes[0, 0].set_title('Global COVID-19 Cases Over Time', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Date', fontsize=12)
axes[0, 0].set_ylabel('Number of Cases', fontsize=12)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Subplot 2: Top 10 countries (FIXED COLUMN NAME)
top10 = df.groupby('Country/Region')['Confirmed'].max().sort_values(ascending=False).head(10)
axes[0, 1].barh(top10.index, top10.values, color='#9b59b6')
axes[0, 1].set_title('Top 10 Countries by Total Cases', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Total Confirmed Cases', fontsize=12)
axes[0, 1].invert_yaxis()
for i, v in enumerate(top10.values):
    axes[0, 1].text(v, i, f' {v:,.0f}', va='center', fontsize=9)

# Subplot 3: Mortality rates by country (FIXED COLUMN NAME)
mortality_by_country = df.groupby('Country/Region')['Mortality_Rate'].mean().sort_values(ascending=False).head(10)
axes[1, 0].bar(range(len(mortality_by_country)), mortality_by_country.values, 
               color='#e67e22')
axes[1, 0].set_title('Top 10 Countries by Mortality Rate', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Country', fontsize=12)
axes[1, 0].set_ylabel('Mortality Rate (%)', fontsize=12)
axes[1, 0].set_xticks(range(len(mortality_by_country)))
axes[1, 0].set_xticklabels(mortality_by_country.index, rotation=45, ha='right')

# Subplot 4: Monthly trends
monthly_data = df.groupby(['Year', 'Month'])['Confirmed'].sum().reset_index()
monthly_data['Period'] = monthly_data['Year'].astype(str) + '-' + monthly_data['Month'].astype(str).str.zfill(2)
axes[1, 1].plot(range(len(monthly_data)), monthly_data['Confirmed'], 
                marker='o', linewidth=2, color='#1abc9c', markersize=4)
axes[1, 1].set_title('Monthly Global Cases Trend', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Month', fontsize=12)
axes[1, 1].set_ylabel('Total Confirmed Cases', fontsize=12)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('covid19_global_trends.png', dpi=300, bbox_inches='tight')
print("✓ Saved: covid19_global_trends.png")
plt.show()

# Figure 2: Distribution Analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('COVID-19 Statistical Distributions', fontsize=18, fontweight='bold')

# Histogram of mortality rates
axes[0].hist(df['Mortality_Rate'].dropna(), bins=50, color='#e74c3c', alpha=0.7, edgecolor='black')
axes[0].set_title('Distribution of Mortality Rates', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Mortality Rate (%)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].axvline(df['Mortality_Rate'].mean(), color='blue', linestyle='--', linewidth=2, label='Mean')
axes[0].legend()

# Boxplot of cases by country (FIXED COLUMN NAME)
latest_data = df[df['Date'] == df['Date'].max()]
top_countries_list = latest_data.nlargest(10, 'Confirmed')['Country/Region'].values
boxplot_data = [df[df['Country/Region'] == country]['Confirmed'].values for country in top_countries_list]
axes[1].boxplot(boxplot_data, labels=top_countries_list)
axes[1].set_title('Cases Distribution - Top 10 Countries', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Number of Cases', fontsize=11)
axes[1].tick_params(axis='x', rotation=45)

# Scatter plot: Cases vs Deaths (FIXED COLUMN NAME)
latest_by_country = df[df['Date'] == df['Date'].max()]
axes[2].scatter(latest_by_country['Confirmed'], latest_by_country['Deaths'], 
                alpha=0.6, s=100, c=latest_by_country['Mortality_Rate'], 
                cmap='YlOrRd', edgecolors='black', linewidth=0.5)
axes[2].set_title('Cases vs Deaths Correlation', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Confirmed Cases', fontsize=11)
axes[2].set_ylabel('Deaths', fontsize=11)
axes[2].grid(True, alpha=0.3)
cbar = plt.colorbar(axes[2].collections[0], ax=axes[2])
cbar.set_label('Mortality Rate (%)', fontsize=10)

plt.tight_layout()
plt.savefig('covid19_distributions.png', dpi=300, bbox_inches='tight')
print("✓ Saved: covid19_distributions.png")
plt.show()

# Figure 3: Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap of COVID-19 Metrics', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('covid19_correlation_heatmap.png', dpi=300, bbox_inches='tight')
print("✓ Saved: covid19_correlation_heatmap.png")
plt.show()

In [ ]:
#SECTION 7: RULE-BASED RISK EVALUATOR
print()
print("RULE-BASED RISK ASSESSMENT SYSTEM")
print()

import numpy as np  # Add this import

# Create Recovery_Rate if it doesn't exist
if 'Recovery_Rate' not in df.columns:
    df['Recovery_Rate'] = df.apply(
        lambda row: (row['Recovered'] / row['Confirmed'] * 100).round(2) 
        if row['Confirmed'] > 0 else 0, 
        axis=1
    )

class COVID19RiskEvaluator:
    def __init__(self):
        self.risk_levels = {
            'Low': {'score': (0, 40), 'color': 'green'},
            'Moderate': {'score': (40, 60), 'color': 'yellow'},
            'High': {'score': (60, 80), 'color': 'orange'},
            'Critical': {'score': (80, 100), 'color': 'red'}
        }
    
    def calculate_cases_per_million(self, total_cases, population):
        return (total_cases / population) * 1_000_000
    
    def calculate_mortality_rate(self, deaths, cases):
        if cases == 0:
            return 0
        return (deaths / cases) * 100
    
    def evaluate_risk(self, total_cases, deaths, population, 
                     test_positivity_rate, hospital_capacity=100):
        risk_score = 0
        
        # Factor 1: Cases per million
        cases_per_million = self.calculate_cases_per_million(total_cases, population)
        if cases_per_million > 10000:
            risk_score += 40
        elif cases_per_million > 5000:
            risk_score += 30
        elif cases_per_million > 2000:
            risk_score += 20
        else:
            risk_score += 10
        
        # Factor 2: Mortality rate
        mortality_rate = self.calculate_mortality_rate(deaths, total_cases)
        if mortality_rate > 4:
            risk_score += 30
        elif mortality_rate > 3:
            risk_score += 20
        elif mortality_rate > 2:
            risk_score += 10
        else:
            risk_score += 5
        
        # Factor 3: Test positivity rate
        if test_positivity_rate > 10:
            risk_score += 20
        elif test_positivity_rate > 5:
            risk_score += 15
        elif test_positivity_rate > 2:
            risk_score += 10
        else:
            risk_score += 5
        
        # Factor 4: Hospital capacity
        if hospital_capacity < 20:
            risk_score += 10
        elif hospital_capacity < 50:
            risk_score += 7
        elif hospital_capacity < 80:
            risk_score += 4
        else:
            risk_score += 0
        
        # Determine risk level
        risk_level = 'Low'
        for level, criteria in self.risk_levels.items():
            if criteria['score'][0] <= risk_score < criteria['score'][1]:
                risk_level = level
                break
        if risk_score >= 80:
            risk_level = 'Critical'
        
        return {
            'risk_score': risk_score,
            'risk_level': risk_level,
            'cases_per_million': round(cases_per_million, 2),
            'mortality_rate': round(mortality_rate, 2),
            'test_positivity_rate': test_positivity_rate,
            'hospital_capacity': hospital_capacity,
            'recommendations': self.get_recommendations(risk_level)
        }
    
    def get_recommendations(self, risk_level):
        recommendations = {
            'Low': ['Continue regular monitoring', 'Maintain basic hygiene practices'],
            'Moderate': ['Increase testing capacity', 'Promote mask usage'],
            'High': ['Implement social distancing', 'Restrict large gatherings'],
            'Critical': ['Implement strict lockdown', 'Emergency healthcare expansion']
        }
        return recommendations.get(risk_level, [])

# Initialize evaluator
evaluator = COVID19RiskEvaluator()

# Example assessments
print("\nExample 1: United States")
print("-" * 60)
usa_assessment = evaluator.evaluate_risk(
    total_cases=5_000_000,
    deaths=180_000,
    population=331_000_000,
    test_positivity_rate=8.5,
    hospital_capacity=65
)
print(f"Risk Score: {usa_assessment['risk_score']}/100")
print(f"Risk Level: {usa_assessment['risk_level']}")

# Batch evaluation for all countries
print("\nBatch Risk Assessment for Top 10 Countries:")
print("=" * 80)

latest = df[df['Date'] == df['Date'].max()]
top10_countries = latest.nlargest(10, 'Confirmed')

risk_results = []
for _, row in top10_countries.iterrows():
    # FIXED: Use correct column name 'Country/Region'
    country = row['Country/Region']
    
    population_map = {
        'US': 331_000_000, 'United States': 331_000_000,
        'India': 1_380_000_000, 'Brazil': 213_000_000,
        'United Kingdom': 68_000_000, 'Russia': 146_000_000,
        'France': 67_000_000, 'Germany': 83_000_000,
        'Italy': 60_000_000, 'Spain': 47_000_000
    }
    
    population = population_map.get(country, 50_000_000)
    test_positivity = np.random.uniform(2, 15)
    
    assessment = evaluator.evaluate_risk(
        total_cases=int(row['Confirmed']),
        deaths=int(row['Deaths']),
        population=population,
        test_positivity_rate=test_positivity
    )
    
    risk_results.append({
        'Country': country,
        'Risk_Level': assessment['risk_level'],
        'Risk_Score': assessment['risk_score'],
        'Cases_per_Million': assessment['cases_per_million']
    })

risk_df = pd.DataFrame(risk_results)
print(risk_df.to_string(index=False))

# Visualize risk levels
plt.figure(figsize=(12, 6))
colors = {'Low': 'green', 'Moderate': 'yellow', 'High': 'orange', 'Critical': 'red'}
bar_colors = [colors[level] for level in risk_df['Risk_Level']]

plt.bar(risk_df['Country'], risk_df['Risk_Score'], color=bar_colors, 
        edgecolor='black', linewidth=1.5, alpha=0.8)
plt.title('COVID-19 Risk Assessment Scores by Country', fontsize=16, fontweight='bold')
plt.xlabel('Country', fontsize=12)
plt.ylabel('Risk Score', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('covid19_risk_scores.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: covid19_risk_scores.png")
plt.show()